In [0]:
################################################################################
# Consider PyStan is you ever move off Databricks and want to stay with Pyton. #
################################################################################
# %pip install PyPI cmdstanpy
# %pip install pymc
# dbutils.library.restartPython()
# %pip list

import numpy as np
import pandas as pd
import scipy.stats as stats
from scipy.stats import beta, binom, uniform
import pymc as pm
import matplotlib.pyplot as plt
import random

# 2.1 p.27
ways = [0, 3, 8, 9, 0]
[item / sum(ways) for item in ways]

# 2.2 p.33
binom.pmf(6, 9, 0.6667)

###########################
# Grid Approximation p.39 #
###########################
# 2.3 p.40
# define grid
p_grid = np.linspace(start=0, stop=1, num=20)
# define prior
prior = np.repeat(1, 20)
# compute likelihood as each value in grid
likelihood = binom.pmf(6, 9, p_grid)
# compute product of likelihood and prior
unstd_posterior = likelihood * prior
# standardize the posterior, so it sums to 1
posterior = unstd_posterior / sum(unstd_posterior)
# 2.4 p.40
plt.plot(p_grid, posterior, linestyle='-', marker='o')
plt.xlabel('probability of water')
plt.ylabel('posterior probability')
plt.title('100 points')
plt.show()
# 2.5 p.40 
p_grid = np.linspace(start=0, stop=1, num=100)
prior = np.where(p_grid < 0.5, 0, 1)
likelihood = binom.pmf(6, 9, p_grid)
unstd_posterior = likelihood * prior
posterior = unstd_posterior / sum(unstd_posterior)
plt.plot(p_grid, posterior, linestyle='-', marker='o')
plt.xlabel('probability of water')
plt.ylabel('posterior probability')
plt.title('100 points')
plt.show()
p_grid = np.linspace(start=0, stop=1, num=100)
prior = np.exp(-5 * abs(p_grid - 0.5 ))
likelihood = binom.pmf(6, 9, p_grid)
unstd_posterior = likelihood * prior
posterior = unstd_posterior / sum(unstd_posterior)
plt.plot(p_grid, posterior, linestyle='-', marker='o')
plt.xlabel('probability of water')
plt.ylabel('posterior probability')
plt.title('100 points')
plt.show()
################################
# Quadratic Approximation p.41 #
################################
# 2.5 p.42 Quadratic Approximation
data = np.repeat((0,1), (6,12)) # 9 tosses, three L (0), six water (1). Also try 18 and 36 tosses.
with pm.Model() as normal_approx:
  p = pm.Uniform('p', 0, 1) # uniform prior
  w = pm.Binomial('w', n=len(data), p=p, observed=data.sum()) #binomial likelihood
  mean_q = pm.find_MAP() # find local maximum a posteriori with this model
  normal_approx.rvs_to_transforms[p] = None # remove transform from the variable `p`
  p_value = normal_approx.rvs_to_values[p]
  p_value.name = p.name # change name so that we can use `mean_q["p"]` value
  std_q = ((1/pm.find_hessian(mean_q, vars=[p]))**0.5)[0] # find the standard deviation
mean_q['p'], std_q
prob = 0.89
z = stats.norm.ppf([(1-prob)/2, (1+prob)/2])
perc = mean_q["p"]+std_q*z
print('Summary of quadratic approximation: mean {0:.2f}, StdDev {1:.2f}, 5.5% CI {2:.2f}, 94.5% CI {3:.2f}'.format(mean_q["p"], std_q[0], perc[0], perc[1]))
# 2.6 p.43 Analytical Calculation
w = 12 # try values of 6, 12, 24
n = 18 # try values of 9, 18, 36
x = np.arange(0, 1, 0.01) # create an x arrary for beta function calc
plt.plot(x, stats.beta.pdf(x, w+1, n-w+1), '--', label='true')
plt.plot(x, stats.norm.pdf(x, mean_q["p"], std_q[0]), label='Quadratic Appx.')
plt.title('n=9')
plt.xlabel('proportion of water')
plt.ylabel('Density')
plt.legend(loc='upper left')

# Course Video https://www.youtube.com/watch?v=R1vcdhPBlXA
# function to toss a globe covered p by water N times
def sim_globe(p=0.7, N=9) : 
  return np.random.choice(["W","L"], size=N, p=[p, 1-p])
sim_globe()
results = pd.DataFrame([sim_globe() for _ in range(10)]) # run 10 simulations
display(results)
# test your code
sim_globe(p=1, N=11) # check that your code produces all water with probability of water equals 100%
sum(np.array(sim_globe(p=0.5, N=int(1e4))) == "W") / 1e4 # check that your code produces simulated results of 50% water
# function to compute posterior distribution
def compute_posterior(the_sample, poss=[0, 0.25, 0.5, 0.75, 1]) : 
  the_sample = np.array(the_sample)
  W = np.sum(the_sample == "W") # number of W observed
  L = np.sum(the_sample == "L") # number of L observed
  ways = np.array([((x*4)**W)*(((1-x)*4)**L) for x in poss])
  post = ways/np.sum(ways)
  df = pd.DataFrame({"poss":poss, "ways":[str(w) for w in ways], "post":np.round(post,3)})
  return df
print(compute_posterior(sim_globe()))
# simulate posterior predictive distribution
post_samples = beta.rvs(a=6+1, b=3+1, size=10000)
pred_post = [np.sum(sim_globe(p, 10) == "W") for p in post_samples]
pred_spec = [np.sum(sim_globe(0.71, 10) == "W") for _ in post_samples]
tab_post = pd.Series(pred_post).value_counts().sort_index()
tab_spec = pd.Series(pred_spec).value_counts().sort_index()
plt.bar(tab_spec.index, tab_spec, color='black', width=0.6, label = 'specific predictive')
plt.bar(tab_post.index, tab_post, color='blue', width=0.2, label ='posterior predictive')
plt.legend(loc='upper left')
plt.xlabel('number of W')
plt.ylabel('count')
plt.show()
# misclassification simulation
def sim_globe_miscls(p=0.71, N=10, x=0.1) :
  true_sample = np.random.choice(["W","L"], size=N, p=[p, 1-p])
  obs_sample = np.array([("L" if ts == "W" else "W") if random.random() < x else ts for ts in true_sample])
  return obs_sample
pred_err_post = [np.sum(sim_globe_miscls(p, 10, 0.1) == "W") for p in post_samples]
tab_err_post = pd.Series(pred_err_post).value_counts().sort_index()
plt.plot(tab_spec.index, tab_spec.values, label='previous posterior', color='black', linewidth=4)
plt.plot(tab_err_post.index, tab_err_post.values, label='misclassification posterior\nerror rate 10%', color='red', linewidth=4)
plt.xlabel("proportion of water")
plt.ylabel("posterior probability")
plt.show()